In [ ]:
# dependencies

import os
import math
import scipy
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from scipy.signal import find_peaks
from scipy import interpolate
from scipy.interpolate import interp1d

from bokeh.plotting import figure, show, output_file, save
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar, ColumnDataSource, DatetimeTickFormatter
from bokeh.palettes import Spectral6

import seaborn as sns

output_notebook()

In [ ]:
### clean event_times data

Start_time = '2021-10-26 17:30:00+0000'
End_time = '2021-10-26 20:30:00+0000'

df = pd.read_csv('/content/drive/MyDrive/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_clean.csv')
df['datetime_concert'] = [pd.to_datetime(Start_time) + pd.to_timedelta(i, unit='sec') for i in df.time]
df.to_csv('/content/drive/MyDrive/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_dt.csv')

In [ ]:
# FinnPy

def nannotime(row,shift,dshift):
    r = row.copy()
    cols = r.index
    for c in cols:
        if not c.startswith('time'):
            if not c.startswith('datetime'):
                r[c] = np.nan
            if c.startswith('datetime'):
                r[c] = r[c] + dshift  
        if c.startswith('time'):
            r[c] = r[c] + shift     
    return r

def gap_nans(data,gap_t):
    # data is a pandas dataframe with columns called time and timestamps which is used to ID gaps greater than gap_T
    # rows of NaN data is added to non-time columns before the first sample, in each gap, and after the last sample of data
    cols = data.columns
    if 'time' in cols:
        time_col = 'time'
    if 'time_concert' in cols:
        time_col = 'time_concert'
    deltat = round(0.35*data[time_col].diff().median())
    dtdeltat =  pd.to_timedelta(deltat,unit = 'ms')
    
    data = data.append(nannotime(data.iloc[-1,:],deltat,dtdeltat),ignore_index=True)
    
    dt = data[time_col].diff()
    a = list(dt[dt>gap_t].index)
    a.sort(reverse=True)
    for gapi in a:
        data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
        data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))

    data = data.append(nannotime(data.iloc[0,:],-1,dtdeltat)).sort_values(time_col,ignore_index=True)
    
    return data

In [ ]:
# 

Start_time = '2021-10-26 17:30:00+0000'
End_time = '2021-10-26 20:30:00+0000'
sample_rate = 50 # Hz
# set standard timestamps in date time and millisecond floats. Datetime for ploting, ms for interp
# 50 Hz
ts_dt = pd.date_range(pd.to_datetime(Start_time), pd.to_datetime(End_time), freq='20ms')
ts_ts = np.arange(pd.to_datetime(Start_time).timestamp(), pd.to_datetime(End_time).timestamp(), 0.02)*1000

dir_devices = '/content/drive/MyDrive/ritmo/CopenhagenMusicLab/Aligned/Hall/'
f_events = '/content/drive/MyDrive/ritmo/CopenhagenMusicLab/Concert_content/Concert_event_times_clean.csv'
file_suffix = 'deviceMotion.csv'

kept_dev = []
kept_df = None

# read events data
df_events = pd.read_csv(f_events)
shift_concert_time = df_events[df_events.time_concert == 0].time.values[0]

# concert time shift
shift_concert_time = df_events[df_events.label == 'tap_60_bpm_1'].time

# go over devices
dev_n = 0
plots = []
index_devices = slice(0, 10) # which devices
list_devices = os.listdir(dir_devices)[index_devices] # devices
print(len(list_devices), ' devices.')

# select event
in_event  = ('claps_on' , 0) # which event
out_event = ('claps_off', 0) # which event

in_event_time = pd.to_datetime(Start_time) + pd.to_timedelta(df_events[df_events.label == in_event[0]].time.to_list()[in_event[1]], unit='sec') # df_events[df_events.label == in_event[0] ].time_concert.to_list()[in_event[1] ] * 10**3
out_event_time = pd.to_datetime(Start_time) + pd.to_timedelta(df_events[df_events.label == out_event[0]].time.to_list()[out_event[1]], unit='sec') # df_events[df_events.label == out_event[0]].time_concert.to_list()[out_event[1]] * 10**3
print('time interval: ', in_event_time, out_event_time)

for f in list_devices:
  print('device: ', dev_n)
  # read data
  df = pd.read_csv(dir_devices + f)
  #print(df.columns)
  df[df.time_concert.between(-100, +100)].to_csv('head_' + f + '.csv')

  # gap_nans
  df['datetime_concert'] = (pd.to_datetime(df['timestamp'], unit='ms'))
  df_gapsafe = gap_nans(df, 100) # 100ms
  df_gapsafe['datetime'] = (pd.to_datetime(df_gapsafe['timestamp'],unit='ms'))

  # resample
  df_resampled = pd.DataFrame(index = ts_dt[:-1])
  cols = df_gapsafe.columns
  for col in cols:
      func = interpolate.interp1d(df_gapsafe['timestamp'], df_gapsafe[col],fill_value='extrapolate')
      df_resampled[col] = func(ts_ts)

  df_resampled.iloc[:100].to_csv('resampled_' + str(dev_n) + '.csv')

  #df_resampled.datetime_concert = pd.to_datetime(df_resampled.datetime_concert)
  #df_resampled.datetime = pd.to_datetime(df_resampled.datetime)

  df_clap = df_resampled[df_resampled.datetime_concert.between(in_event_time.timestamp() * 10**9, out_event_time.timestamp() * 10**9)]
  df.datetime_concert = pd.to_datetime(df.datetime_concert).astype('int64')
  df_clap_old = df[df.datetime_concert.between(in_event_time.timestamp() * 10**9, out_event_time.timestamp() * 10**9)]
  #print(in_event_time_concert.timestamp(), out_event_time_concert.timestamp())
  #print(df_resampled.datetime_concert)

  aud_df = pd.DataFrame(index = df_clap.index)
  aud_df = aud_df.loc[in_event_time : out_event_time]

  # norm
  norm = np.linalg.norm(df_clap[['x','y','z']].diff().values,axis=1)
  if np.nanmedian(norm)>0:
      aud_df[str(dev_n)] = norm/np.nanmedian(norm)
  else:
      aud_df[str(dev_n)] = norm/np.nanmean(norm)

  # norm OLD
  aud_df_old = pd.DataFrame(index = df_clap_old.index)
  norm = np.linalg.norm(df_clap_old[['x','y','z']].diff().values,axis=1)
  if np.nanmedian(norm)>0:
      aud_df_old[str(dev_n)] = norm/np.nanmedian(norm)
  else:
      aud_df_old[str(dev_n)] = norm/np.nanmean(norm)
  aud_df_old['time'] = pd.to_datetime(df_clap_old.datetime_concert)

  # rms step 1
  i = 0
  window_size = sample_rate # / 10
  rms_x = []
  rms_y = []
  while True:
    if i >= df_clap.acc_1d.shape[0]:
      break
    in_window  = int(i-window_size/2) if i>window_size/2 else 0
    out_window = int(i+window_size/2) if df_clap.acc_1d.shape[0] - i > window_size/2 else df_clap.acc_1d.shape[0]
    rms_value = rms(df_clap.acc_1d.iloc[in_window:out_window])
    rms_time  = df_clap.time_concert.iloc[i] / 10**3 + shift_concert_time
    rms_y.append(rms_value)
    rms_x.append(rms_time)
    i = i + 1
  #print(rms_series[:100])
  

  """
  p = figure(title = 'Claps', plot_width=1250, plot_height=250, x_axis_type='datetime', sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
  p.line(aud_df.index, aud_df.acc_1d)
  print(aud_df.index, aud_df.acc_1d)
  p.xaxis.formatter = DatetimeTickFormatter()
  p.xaxis.ticker.desired_num_ticks = 30

  # crop section
  in_event  = ('claps_on', 0) # which event
  out_event = ('claps_off', 0) # which event

  in_event_index  = df_events.index[df_events.label == in_event[0] ].to_list()[in_event[1] ]
  out_event_index = df_events.index[df_events.label == out_event[0]].to_list()[out_event[1]]

  in_event_sec  = df_events.iloc[in_event_index ].time * 10**3 # ms
  out_event_sec = df_events.iloc[out_event_index].time * 10**3 # ms
  # out_event_sec = in_event_sec + 1000 # interval

  in_event_dt  = pd.to_datetime(Start_time) + pd.to_timedelta(in_event_sec , 'ms')
  out_event_dt = pd.to_datetime(Start_time) + pd.to_timedelta(out_event_sec, 'ms')

  df_clap = aud_df.loc[in_event_dt : out_event_dt]
  print(df_clap.head())
  """

  """
  #count peaks
  prev = 0
  cnt = 0
  peak = False
  for i in norm_processed:
    if i > mid_range and not peak:
      peak = True
      cnt = cnt + 1
    elif i < mid_range and peak:
      peak = False
    prev = i
  """

  # peaks
  #peak_index_list = scipy.signal.find_peaks(aud_df['acc_1d'], threshold=1, distance=10)[0]
  #df_peaks = aud_df.iloc[peak_index_list]
  #print('peaks: ', df_peaks.shape[0])

  # time axis
  #time_clap  = [(i / 10**3 + shift_concert_time) for i in df_clap.index.to_list()]
  #time_peaks = [(i / 10**3 + shift_concert_time) for i in df_peaks.index.to_list()]

  # figure
  #s = figure(title = 'Claps', plot_width=1250, plot_height=250, sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
  #s.line(df_clap.index, df_clap.acc_1d)
  #s.scatter(time_peaks, df_peaks.acc_1d, color='red', legend_label = str(df_peaks.shape[0]) + ' claps')
  #s.xaxis.ticker.desired_num_ticks = int(df_clap.index.shape[0] / 60)
  #s.xaxis.major_label_orientation = math.pi/2
  #plots.append(p)
  
  """
  p = figure(title = 'Claps Resampled', plot_width=1250, plot_height=200, x_axis_type='datetime', sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
  p.line(x='index', y=str(dev_n), source=aud_df)
  p.xaxis.ticker.desired_num_ticks = 30
  plots.append(p)
  p = figure(title = 'Claps', plot_width=1250, plot_height=200, x_axis_type='datetime', sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
  p.line(x='time', y=str(dev_n), source=aud_df_old, line_color='red')
  p.xaxis.ticker.desired_num_ticks = 30
  plots.append(p)
  """

  dev_n += 1
  
#chart = sns.heatmap(data=aud_df.transpose(), ax=ax,vmin=0, vmax=5,cbar=False)

show(column(plots))
#aud_df

NameError: ignored

In [ ]:
aud_df

,0
2021-10-26 17:45:46.020000+00:00,NaN
2021-10-26 17:45:46.040000+00:00,1.786842
2021-10-26 17:45:46.060000+00:00,0.807655
2021-10-26 17:45:46.080000+00:00,0.854080
2021-10-26 17:45:46.100000+00:00,1.120151
...,...
2021-10-26 17:46:03.040000+00:00,2.264970
2021-10-26 17:46:03.060000+00:00,0.345054
2021-10-26 17:46:03.080000+00:00,1.105619
2021-10-26 17:46:03.100000+00:00,1.211088


In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

In [ ]:
def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # get coefficients
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

In [ ]:
# Filter adjustments
T = 10.0        # Sample Period
fs = 50.0       # Hz
cutoff = 12.5      # cutoff frequency of the filter # Hz
nyq = 0.5 * fs  # Nyquist Frequency ???
order = 2 
n = int(T * fs)

plots = []
data_raw = aud_df['0'].fillna(0)
data_filtered = butter_lowpass_filter(data_raw, cutoff, fs, order)
aud_df['filtered'] = data_filtered

p = figure(title = 'Filtered', plot_width=1250, plot_height=200, sizing_mode='stretch_width', tools='pan,wheel_zoom,box_zoom,reset')
p.line(x='index', y='0', source=aud_df, line_alpha=0.5)
p.line(x='index', y='filtered', source=aud_df, color='red')
p.xaxis.ticker.desired_num_ticks = 30
plots.append(p)

show(column(plots))
